## MSDI Chroma Feature Analysis

In [ ]:
import pandas as pd
import numpy as np
import tables
import h5py
import pickle

import math
import random

import os, sys, glob
from pathlib import Path

import keras
from keras import layers
from keras import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, Dropout, MaxPooling2D

import tensorflow as tf

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import requests

def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 3276800

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://drive.google.com/u/0/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    




In [ ]:
# TAKE ID FROM SHAREABLE LINK
#https://drive.google.com/file/d/1wduOo4DLWGEAF44odjv6BQlsxYsO_0c6/view?usp=sharing
file_id = r'1wduOo4DLWGEAF44odjv6BQlsxYsO_0c6'
# DESTINATION FILE ON YOUR DISK
destination = 'shortlisted_tracks_df.pkl' #pickle4
download_file_from_google_drive(file_id, destination)

In [ ]:
# TAKE ID FROM SHAREABLE LINK
#https://drive.google.com/file/d/1dhgh0ZPptiqBFHZ1vPhBU6-GH39k2NPY/view?usp=sharing
file_id = r'1dhgh0ZPptiqBFHZ1vPhBU6-GH39k2NPY'
# DESTINATION FILE ON YOUR DISK
destination = 'msd_data.zip'
download_file_from_google_drive(file_id, destination)

In [ ]:
!unzip -q msd_data.zip

Streaming output truncated to the last 5000 lines.
  inflating: msd_data/L/Q/K/TRLQKZS128F4247F58.h5  
   creating: msd_data/L/Q/L/
  inflating: msd_data/L/Q/L/TRLQLIT128F429095C.h5  
  inflating: msd_data/L/Q/L/TRLQLZB12903CC2383.h5  
   creating: msd_data/L/Q/P/
  inflating: msd_data/L/Q/P/TRLQPZH128F92C46D5.h5  
   creating: msd_data/L/Q/I/
  inflating: msd_data/L/Q/I/TRLQIJS128F92D9533.h5  
   creating: msd_data/L/U/
   creating: msd_data/L/U/R/
  inflating: msd_data/L/U/R/TRLUROI128F92FFBFD.h5  
   creating: msd_data/L/U/C/
  inflating: msd_data/L/U/C/TRLUCQX128F428C72E.h5  
   creating: msd_data/L/U/L/
  inflating: msd_data/L/U/L/TRLULKS12903CEF56C.h5  
   creating: msd_data/L/U/X/
  inflating: msd_data/L/U/X/TRLUXAC128F4218F2F.h5  
   creating: msd_data/L/X/
   creating: msd_data/L/X/Y/
  inflating: msd_data/L/X/Y/TRLXYTL128F427853A.h5  
   creating: msd_data/L/X/T/
  inflating: msd_data/L/X/T/TRLXTNA128F933B51A.h5  
   creating: msd_data/L/X/O/
  inflating: msd_data/L/X/O/TRLXO

In [ ]:
with open('shortlisted_tracks_df' +'.pkl', 'rb') as handle:
    shortlisted_tracks_df = pickle.load(handle)

In [ ]:
shortlisted_tracks_df

,Genre
TrackId,
TREQVVI128F427C38E,Country
TRYCYSD128F148CF20,Country
TRLCZET12903D03F70,Country
TRJORPT128F42BAA8D,Country
TROBJYA128F42A2984,Country
...,...
TRQFZEO128E07930BC,Rock
TRNSWUN128F4247F87,Rock
TRLENOU128F4281E94,Rock


In [ ]:
track_id = 'TRAAHSY128F147BB5C'
base_dir = 'msd_data/' + track_id[2] + '/' + track_id[3] + '/' + track_id[4]
filepath = base_dir + '/' + track_id + '.h5'

hfile = h5py.File(filepath, 'r')

hfile.keys()

<KeysViewHDF5 ['analysis', 'metadata', 'musicbrainz']>

In [ ]:
hfile['analysis'].keys()

<KeysViewHDF5 ['bars_confidence', 'bars_start', 'beats_confidence', 'beats_start', 'sections_confidence', 'sections_start', 'segments_confidence', 'segments_loudness_max', 'segments_loudness_max_time', 'segments_loudness_start', 'segments_pitches', 'segments_start', 'segments_timbre', 'songs', 'tatums_confidence', 'tatums_start']>

In [ ]:
hfile['analysis']['segments_start'].shape[0]

728

In [ ]:
hfile['analysis']['segments_pitches'][0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
!pip install librosa

In [ ]:
import librosa
import librosa.display

sample_index = 0

track_id = shortlisted_tracks_df.index.values[sample_index]
genre = shortlisted_tracks_df.iloc[sample_index][0]

filepath = 'msd_data/' + track_id[2] + '/' + track_id[3] + '/' + track_id[4] + '/' + track_id + '.h5'
hfile = h5py.File(filepath, 'r')

print ('chroma plot for track_id=', track_id, " genre=", genre, " is: ")

pitch_values = np.array(hfile['analysis']['segments_pitches'])
librosa.display.specshow(pitch_values, x_axis='time', y_axis='chroma', hop_length=12)

hfile.close()

In [ ]:
import librosa
import librosa.display

sample_index = 700

track_id = shortlisted_tracks_df.index.values[sample_index]
genre = shortlisted_tracks_df.iloc[sample_index][0]

filepath = 'msd_data/' + track_id[2] + '/' + track_id[3] + '/' + track_id[4] + '/' + track_id + '.h5'
hfile = h5py.File(filepath, 'r')

print ('chroma plot for track_id=', track_id, " genre=", genre, " is: ")

pitch_values = np.array(hfile['analysis']['segments_pitches'])
librosa.display.specshow(pitch_values, x_axis='time', y_axis='chroma', hop_length=12)

hfile.close()

In [ ]:
import librosa
import librosa.display

sample_index = 1400

track_id = shortlisted_tracks_df.index.values[sample_index]
genre = shortlisted_tracks_df.iloc[sample_index][0]

filepath = 'msd_data/' + track_id[2] + '/' + track_id[3] + '/' + track_id[4] + '/' + track_id + '.h5'
hfile = h5py.File(filepath, 'r')

print ('chroma plot for track_id=', track_id, " genre=", genre, " is: ")

pitch_values = np.array(hfile['analysis']['segments_pitches'])
librosa.display.specshow(pitch_values, x_axis='time', y_axis='chroma', hop_length=12)

hfile.close()

In [ ]:
import librosa
import librosa.display

sample_index = 2100

track_id = shortlisted_tracks_df.index.values[sample_index]
genre = shortlisted_tracks_df.iloc[sample_index][0]

filepath = 'msd_data/' + track_id[2] + '/' + track_id[3] + '/' + track_id[4] + '/' + track_id + '.h5'
hfile = h5py.File(filepath, 'r')

print ('chroma plot for track_id=', track_id, " genre=", genre, " is: ")

pitch_values = np.array(hfile['analysis']['segments_pitches'])
librosa.display.specshow(pitch_values, x_axis='time', y_axis='chroma', hop_length=12)

hfile.close()

In [ ]:
import librosa
import librosa.display

sample_index = 2800

track_id = shortlisted_tracks_df.index.values[sample_index]
genre = shortlisted_tracks_df.iloc[sample_index][0]

filepath = 'msd_data/' + track_id[2] + '/' + track_id[3] + '/' + track_id[4] + '/' + track_id + '.h5'
hfile = h5py.File(filepath, 'r')

print ('chroma plot for track_id=', track_id, " genre=", genre, " is: ")

pitch_values = np.array(hfile['analysis']['segments_pitches'])
librosa.display.specshow(pitch_values, x_axis='time', y_axis='chroma', hop_length=12)

hfile.close()

In [ ]:
import librosa
import librosa.display

sample_index = 3500

track_id = shortlisted_tracks_df.index.values[sample_index]
genre = shortlisted_tracks_df.iloc[sample_index][0]

filepath = 'msd_data/' + track_id[2] + '/' + track_id[3] + '/' + track_id[4] + '/' + track_id + '.h5'
hfile = h5py.File(filepath, 'r')

print ('chroma plot for track_id=', track_id, " genre=", genre, " is: ")

pitch_values = np.array(hfile['analysis']['segments_pitches'])
librosa.display.specshow(pitch_values, x_axis='time', y_axis='chroma', hop_length=12)

hfile.close()

In [ ]:
import librosa
import librosa.display

sample_index = 4200

track_id = shortlisted_tracks_df.index.values[sample_index]
genre = shortlisted_tracks_df.iloc[sample_index][0]

filepath = 'msd_data/' + track_id[2] + '/' + track_id[3] + '/' + track_id[4] + '/' + track_id + '.h5'
hfile = h5py.File(filepath, 'r')

print ('chroma plot for track_id=', track_id, " genre=", genre, " is: ")

pitch_values = np.array(hfile['analysis']['segments_pitches'])
librosa.display.specshow(pitch_values, x_axis='time', y_axis='chroma', hop_length=12)

hfile.close()

In [ ]:
track_count = len(shortlisted_tracks_df.index.values)
seg_count = 65
pitch_count = 12
min_segments = 728
max_segments = 0

msdi_mfcc_values = np.zeros((track_count, seg_count, pitch_count), dtype=np.float32)

for i, track_id in enumerate(shortlisted_tracks_df.index.values):
    base_dir = 'msd_data/' + track_id[2] + '/' + track_id[3] + '/' + track_id[4]
    filepath = base_dir + '/' + track_id + '.h5'
    try:
      hfile = h5py.File(filepath, 'r')
      num_segments = hfile['analysis']['segments_start'].shape[0]
      if num_segments < min_segments:
        min_segments = num_segments

      if num_segments > max_segments:
        max_segments = num_segments

      mfcc_tmp = np.array(hfile['analysis']['segments_pitches'])[:65]
      msdi_mfcc_values[i] = mfcc_tmp

    except (RuntimeError, TypeError, NameError, OSError):
      print('failed for file: ', filepath)

failed for file:  msd_data/V/J/P/TRVJPNB128F42967E4.h5
failed for file:  msd_data/M/S/T/TRMSTPF12903CBF7F2.h5
failed for file:  msd_data/F/H/Q/TRFHQTD12903CFD700.h5
failed for file:  msd_data/S/J/A/TRSJAHY128F932B5E5.h5
failed for file:  msd_data/H/H/A/TRHHAEJ128F148B850.h5
failed for file:  msd_data/J/R/W/TRJRWOF128F147180D.h5


In [ ]:
track_count = len(shortlisted_tracks_df.index.values)
seg_count = 3072
mfcc_count = 12

msdi_mfcc_values = np.zeros((track_count, seg_count, mfcc_count), dtype=np.float32)

for i, track_id in enumerate(shortlisted_tracks_df.index.values):
    filepath = 'msd_data/' + track_id[2] + '/' + track_id[3] + '/' + track_id[4] + '/' + track_id + '.h5'
    try:
      hfile = h5py.File(filepath, 'r')
      num_segments = hfile['analysis']['segments_start'].shape[0]
      msdi_mfcc_values[i][:num_segments] = np.array(hfile['analysis']['segments_pitches'])
      hfile.close()

    except (RuntimeError, TypeError, NameError, OSError):
      print('failed for file: ', filepath)

In [ ]:
min_segments

65

In [ ]:
max_segments

In [ ]:
msdi_mfcc_values[2].max()

1.0

In [ ]:
msdi_mfcc_values[2].min()

-1.0

In [ ]:
shortlisted_tracks_df['Genre'] = shortlisted_tracks_df['Genre'].astype('category')

In [ ]:
genre_counts = 7
msdi_genres = tf.keras.utils.to_categorical(shortlisted_tracks_df['Genre'].cat.codes, genre_counts)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(msdi_mfcc_values, msdi_genres, random_state=1)

In [ ]:
module_suffix = '_pitch'
genre_counts = 7

In [ ]:
#Defining a function to save the objects as a pickle file
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
save_obj(x_train, 'x_train' + module_suffix)
save_obj(y_train, 'y_train' + module_suffix)

In [ ]:
save_obj(x_test, 'x_test' + module_suffix)
save_obj(y_test, 'y_test' + module_suffix)

In [ ]:
#'''
with open('x_train' + module_suffix +'.pkl', 'rb') as handle:
    x_train = pickle.load(handle)

with open('y_train' + module_suffix +'.pkl', 'rb') as handle:
    y_train = pickle.load(handle)

#'''

"\nwith open('x_train' + module_suffix +'.pkl', 'rb') as handle:\n    x_train = pickle.load(handle)\n\nwith open('y_train' + module_suffix +'.pkl', 'rb') as handle:\n    y_train = pickle.load(handle)\n\n#"

In [ ]:
#'''
with open('x_test' + module_suffix +'.pkl', 'rb') as handle:
    x_test = pickle.load(handle)

with open('y_test' + module_suffix +'.pkl', 'rb') as handle:
    y_test = pickle.load(handle)

#'''

"\nwith open('x_test' + module_suffix +'.pkl', 'rb') as handle:\n    x_test = pickle.load(handle)\n\nwith open('y_test' + module_suffix +'.pkl', 'rb') as handle:\n    y_test = pickle.load(handle)\n\n#"

In [ ]:
x_train.shape

(3675, 65, 12)

In [ ]:
y_train.shape

(3675, 7)

In [ ]:
x_train[np.isnan(x_train)] = 0
x_test[np.isnan(x_test)] = 0

In [ ]:
x_train = x_train.reshape(x_train.shape[0], seg_count, pitch_count, 1)
x_test = x_test.reshape(x_test.shape[0], seg_count, pitch_count, 1)

In [ ]:
#x_train.shape

In [ ]:
from keras.layers import Dense, Activation, Flatten, Conv1D, Dropout, MaxPooling1D

model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 1), input_shape=x_train[0].shape, activation='relu', kernel_initializer='he_uniform'))
#model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3, 1), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling2D(pool_size=(2,1)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.5))

model.add(Dense(genre_counts, activation='softmax', kernel_initializer='he_uniform'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
model.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7fb899d200d0>>

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)
history = model.fit(x_train, y_train, epochs=20, batch_size=64, callbacks=[callback], verbose=1, validation_data=(x_test, y_test))

Epoch 1/50
58/58 [==============================] - 11s 182ms/step - loss: 3.1519 - accuracy: 0.1461 - val_loss: 1.9370 - val_accuracy: 0.1388
Epoch 2/50
58/58 [==============================] - 10s 175ms/step - loss: 1.9406 - accuracy: 0.1605 - val_loss: 1.9262 - val_accuracy: 0.1600
Epoch 3/50
58/58 [==============================] - 10s 177ms/step - loss: 1.9282 - accuracy: 0.1644 - val_loss: 1.9284 - val_accuracy: 0.1600
Epoch 4/50
58/58 [==============================] - 10s 176ms/step - loss: 1.9020 - accuracy: 0.1869 - val_loss: 1.8863 - val_accuracy: 0.1967
Epoch 5/50
58/58 [==============================] - 10s 177ms/step - loss: 1.8774 - accuracy: 0.2046 - val_loss: 1.8844 - val_accuracy: 0.2024
Epoch 6/50
58/58 [==============================] - 10s 177ms/step - loss: 1.8547 - accuracy: 0.2201 - val_loss: 1.8633 - val_accuracy: 0.2261
Epoch 7/50
58/58 [==============================] - 10s 177ms/step - loss: 1.8062 - accuracy: 0.2520 - val_loss: 1.8541 - val_accuracy: 0.2343